In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
data = pd.read_csv('../data/raw_postpro.csv', encoding = 'cp949')
# 컬럼 삭제
df = data.drop(['청구서번호','No.',  '선박입고','완료 여부','리드타임_음수제거','청구량','견적','견적수량','견적화폐','견적단가','발주번호','발주','발주수량','발주금액','미입고 기간','리드타임','창고입고','창고입고수량','입고창고','창고출고','창고출고수량','출고선박','출고운반선','선박입고','선박입고수량','완료 여부'], axis=1)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Subject      20517 non-null  object
 1   Machinery    20517 non-null  object
 2   Assembly     20517 non-null  object
 3   청구품목         20517 non-null  object
 4   Part No.1    20517 non-null  object
 5   Part No.2    20517 non-null  object
 6   key1         20517 non-null  object
 7   key2         20517 non-null  object
 8   발주처          20517 non-null  object
 9   D/T          20517 non-null  object
 10  Control No.  20517 non-null  object
dtypes: object(11)
memory usage: 1.7+ MB


In [4]:
df = df[['청구품목','발주처','Machinery', 'Assembly' , "Part No.1", "Part No.2",'key2']]
# 'Machinery', 'Assembly', '청구품목', 'Part No.1', 'Part No.2', 'key1', '발주처'

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   청구품목       20517 non-null  object
 1   발주처        20517 non-null  object
 2   Machinery  20517 non-null  object
 3   Assembly   20517 non-null  object
 4   Part No.1  20517 non-null  object
 5   Part No.2  20517 non-null  object
 6   key2       20517 non-null  object
dtypes: object(7)
memory usage: 1.1+ MB


In [6]:
from sklearn import preprocessing
label_encoders = {}  # 각 열에 대한 LabelEncoder를 저장하기 위한 딕셔너리
columns_to_encode = ['key2']  # 인코딩을 수행할 열의 이름 리스트

for column in columns_to_encode:
    le = preprocessing.LabelEncoder()
    le.fit(df[column])
    label_encoders[column] = le # 딕셔너리에 저장
    df[column+"_encoded"] = le.transform(df[column]) # 새로운 encoding 된 컬럼 추가

In [7]:
df = df.drop(['key2'], axis=1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   청구품목          20517 non-null  object
 1   발주처           20517 non-null  object
 2   Machinery     20517 non-null  object
 3   Assembly      20517 non-null  object
 4   Part No.1     20517 non-null  object
 5   Part No.2     20517 non-null  object
 6   key2_encoded  20517 non-null  int32 
dtypes: int32(1), object(6)
memory usage: 1.0+ MB


In [9]:
# df_desc = df[[ '청구품목', '발주처']].apply(lambda row: ' '.join(row), axis=1)
df_desc = df[[ '청구품목', '발주처','Machinery', 'Assembly' , "Part No.1", "Part No.2"]].apply(lambda row: ' '.join(row), axis=1)

In [10]:
lis = df_desc.values

In [11]:
cnt = 0
for i in lis:
    print(i)
    cnt += 1
    if cnt == 20:
        break

SEAL-O-RING-STOR HAEIN Coporation_Cheonan NO.1 GENERATOR ENGINE 323-6480 LINES GP-FUEL  7.00E-275 0
OIL COOLER & LINES HAEIN Coporation_Cheonan NO.2 GENERATOR ENGINE GASKET KIT 7.00E-275 0
WASHER HAEIN Coporation_Cheonan NO.2 GENERATOR ENGINE 285-8374 MANIFOLD GP-EXH  7.00E-275 0
BOLT-HIGH TEMP HAEIN Coporation_Cheonan NO.1 GENERATOR ENGINE 159-8828 TURBO GP 7.00E-275 0
SEAL HAEIN Coporation_Cheonan NO.1 GENERATOR ENGINE 2N4727 INSTRUMNT PANEL GP 7.00E-275 0
CORE CHARGES FOR CYLINDER PACK AS HAEIN Coporation_Cheonan NO.2 GENERATOR ENGINE 8N-6224 PISTON GP-ROD& 7.00E-275 0
PUMP GP-F TFR-REMAN HAEIN Coporation_Cheonan NO.3 GENERATOR ENGINE 8N-6151 PUMP GP-F TFR  7.00E-275 0
GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.1 GENERATOR ENGINE 8N-7174 GEAR GP-FRONT  7.00E-275 0
GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.3 GENERATOR ENGINE 8N-7174 GEAR GP-FRONT  7.00E-275 0
GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.3 GENERATOR ENGINE 8N-7174 GEAR GP-FRONT  7.00E-275 0
GEAR-WTR PUMP DR HAEIN

In [11]:
# vocab_list = []
# for words in df_desc.str.split():
#   if isinstance(words, list):
#     #print(type(words), words)
#     vocab_list = vocab_list + words
# len(vocab_list)

# 단어갯수

In [12]:
desc = df_desc.to_frame()

In [13]:
desc['key2'] = df['key2_encoded']

In [14]:
desc

,0,key2
0,SEAL-O-RING-STOR HAEIN Coporation_Cheonan NO.1...,18
1,OIL COOLER & LINES HAEIN Coporation_Cheonan NO...,18
2,WASHER HAEIN Coporation_Cheonan NO.2 GENERATOR...,18
3,BOLT-HIGH TEMP HAEIN Coporation_Cheonan NO.1 G...,18
4,SEAL HAEIN Coporation_Cheonan NO.1 GENERATOR E...,18
...,...,...
20512,HYD. CYLINDER_NET BOAT DAVIT MARINE HYDROTEC C...,20
20513,HYD. CYLINDER_N.D CRANE_BOOM_LIFTER MARINE HYD...,20
20514,"PULLY PORT RELIEF ENGINEERING CO.,LTD. 6M NET ...",5
20515,"CYLINDER PORT RELIEF ENGINEERING CO.,LTD. 7M ...",20


In [15]:
desc.columns = ['origin','key2']

In [16]:
desc.head()

,origin,key2
0,SEAL-O-RING-STOR HAEIN Coporation_Cheonan NO.1...,18
1,OIL COOLER & LINES HAEIN Coporation_Cheonan NO...,18
2,WASHER HAEIN Coporation_Cheonan NO.2 GENERATOR...,18
3,BOLT-HIGH TEMP HAEIN Coporation_Cheonan NO.1 G...,18
4,SEAL HAEIN Coporation_Cheonan NO.1 GENERATOR E...,18


In [17]:
X = desc['origin'].values

In [18]:
y = desc['key2'].values

In [19]:
X,y

(array(['SEAL-O-RING-STOR HAEIN Coporation_Cheonan NO.1 GENERATOR ENGINE 323-6480 LINES GP-FUEL  7.00E-275 0',
        'OIL COOLER & LINES HAEIN Coporation_Cheonan NO.2 GENERATOR ENGINE GASKET KIT 7.00E-275 0',
        'WASHER HAEIN Coporation_Cheonan NO.2 GENERATOR ENGINE 285-8374 MANIFOLD GP-EXH  7.00E-275 0',
        ...,
        'PULLY PORT RELIEF ENGINEERING CO.,LTD. 6M NET BOAT ELECTRIC CLUTCH Φ305X5V-3 0',
        'CYLINDER  PORT RELIEF ENGINEERING CO.,LTD. 7M NET BOAT HAND HYDRAULIC STEERING SYSTEM Φ50 x S195 CYLINDER 0',
        'SEAL KIT FOR CYLINDER PORT RELIEF ENGINEERING CO.,LTD. 7M NET BOAT HAND HYDRAULIC STEERING SYSTEM Φ50 x S195 CYLINDER SEAL KIT 0'],
       dtype=object),
 array([18, 18, 18, ...,  5, 20, 20]))

In [20]:
# 학습에 사용할 단어장
tokenizer = tf.keras.preprocessing.text.Tokenizer()

# 토큰화
tokenizer.fit_on_texts(X)
print(len(tokenizer.word_index))

12218


In [21]:
x = tokenizer.texts_to_sequences(X)

In [22]:
cnt = [len(i) for i in x ]

In [23]:
max(cnt), sum(cnt)/len(cnt)

(35, 17.162938051372034)

In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_X = pad_sequences(x,30)

In [25]:
padded_X.shape

(20517, 30)

In [26]:
# 총 단어 갯수
word_size = len(tokenizer.word_index) +1

In [39]:
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, concatenate, Flatten, Conv1D, GlobalMaxPooling1D, LSTM

model = Sequential()
model.add(Embedding(word_size, 20, input_length=30)) 
model.add(LSTM(128, return_sequences=True))  # LSTM 레이어 추가
model.add(Flatten())
model.add(Dense(64, activation='relu', name='Dense_output'))
model.add(Dense(61, activation='softmax', name="Prediction_output"))

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 30, 20)            244380    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           76288     
                                                                 
 flatten_1 (Flatten)         (None, 3840)              0         
                                                                 
 Dense_output (Dense)        (None, 64)                245824    
                                                                 
 Prediction_output (Dense)   (None, 61)                3965      
                                                                 
Total params: 570,457
Trainable params: 570,457
Non-trainable params: 0
_________________________________________________________________


In [41]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
# optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
# optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.001)
# optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.01)
# optimizer = tf.keras.optimizers.Adamax(learning_rate=0.01)

In [42]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                       optimizer=optimizer,
                       metrics=['accuracy'])

In [43]:
# 데이터를 학습용과 검증용으로 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_X, y, test_size=0.3, random_state=42,shuffle=True)

In [44]:
# 딥러닝 모델을 만들기 위해 categorical 함수 사용
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train, num_classes=61)
y_test_categorical = to_categorical(y_test, num_classes=61)

In [45]:
model.fit(X_train,y_train_categorical, epochs= 20, batch_size=32)

Epoch 1/20
449/449 [==============================] - 4s 6ms/step - loss: 1.7786 - accuracy: 0.5680
Epoch 2/20
449/449 [==============================] - 3s 6ms/step - loss: 0.5001 - accuracy: 0.8777
Epoch 3/20
449/449 [==============================] - 3s 6ms/step - loss: 0.1821 - accuracy: 0.9508
Epoch 4/20
449/449 [==============================] - 2s 6ms/step - loss: 0.1038 - accuracy: 0.9728
Epoch 5/20
449/449 [==============================] - 3s 6ms/step - loss: 0.0935 - accuracy: 0.9748
Epoch 6/20
449/449 [==============================] - 3s 6ms/step - loss: 0.0940 - accuracy: 0.9757
Epoch 7/20
449/449 [==============================] - 3s 6ms/step - loss: 0.0877 - accuracy: 0.9783
Epoch 8/20
449/449 [==============================] - 3s 6ms/step - loss: 0.0613 - accuracy: 0.9858
Epoch 9/20
449/449 [==============================] - 3s 6ms/step - loss: 0.0290 - accuracy: 0.9924
Epoch 10/20
449/449 [==============================] - 3s 6ms/step - loss: 0.0685 - accuracy: 0.9873

In [46]:
y_pred= model.predict(X_test)

193/193 [==============================] - 1s 2ms/step


In [47]:
# y_pred 랑 y_test 를 원래 labeling 된 데이터로 변환
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test_categorical, axis=1)

In [48]:
y_pred_classes[:100],y_test_classes[:100]

(array([54, 33, 24, 45, 24, 57, 41, 10,  4, 41, 57, 20, 26, 50, 26, 24, 56,
        57, 40, 30, 40, 57,  8, 38, 24,  4, 46, 37, 57, 56, 24, 24,  4,  4,
        51, 40,  0,  6, 58, 24, 26, 45, 19, 50, 24,  4, 30,  4,  7,  4, 39,
        46, 56,  7, 20, 33, 34, 25, 58, 10, 12, 48, 24, 56, 24, 24, 37,  4,
        33, 40, 22,  8, 23, 44, 45, 20, 40,  8, 56, 42, 40, 24, 24, 45, 20,
         7, 26, 24, 46, 42, 24, 18, 45, 40, 24, 45, 58, 40, 12, 19],
       dtype=int64),
 array([54, 33, 24, 45, 24, 57, 41, 10,  4, 41, 57, 20, 26, 41, 26, 24, 56,
        57, 40, 30, 40, 57,  8, 38, 24,  4, 46, 37, 57, 56, 24, 24,  4,  4,
        51, 40,  0,  6, 58, 24, 26, 42, 19, 50, 24, 48, 10,  4,  7,  4, 39,
        46, 56,  7, 20, 33, 34, 25, 58, 10, 12, 48, 24, 56, 24, 24, 37,  4,
        33, 40, 22,  8, 23, 44, 45, 20, 40,  8, 56, 42, 40, 24, 24, 45, 20,
        36, 26, 24, 46, 42, 24, 18, 45, 40, 40, 24, 58, 19, 24, 26],
       dtype=int64))

In [49]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score
f1 = f1_score(y_pred_classes, y_test_classes, average='macro')
f1

0.7565184322107507

In [ ]:
# 단어장을 이용해 문장을 숫자로 변경
encoded_text=tokenizer.texts_to_sequences(pre_desc['split_desc'])
pre_desc['encode_desc'] = tokenizer.texts_to_sequences(pre_desc['split_desc'])

In [ ]:
len_list = pre_desc['encode_desc'].apply(lambda x: len(x))
max(len_list), sum(len_list)/len(len_list)

In [ ]:
top_words = 1000
max_len = 10
vector_len = 30

In [ ]:
# OOV: Out-Of-Vocabulary
# 사전에서 빈도수 높은 단어 1,000 개만 사용
# 사전에 없는 단어 <OOV> 처리
tokenizer2 = tf.keras.preprocessing.text.Tokenizer(num_words=top_words, oov_token='<OOV>')
tokenizer2.fit_on_texts(pre_desc['split_desc'])
# print('tokenizer2.index_docs', tokenizer2.index_docs)
# print('tokenizer2.index_word', tokenizer2.index_word)
# print('tokenizer2.word_index', tokenizer2.word_index)
pre_desc['encode_desc_1000'] = tokenizer2.texts_to_sequences(pre_desc['split_desc'])

In [ ]:
pre_desc.head(10)

In [ ]:
# 문장의 길이가 다르므로 일치
encode_desc_padding = tf.keras.preprocessing.sequence.pad_sequences(pre_desc['encode_desc_1000'], maxlen=max_len)

In [ ]:
encode_desc_padding.shape

In [ ]:
encode_desc_padding[:5]

In [ ]:
embedding = tf.keras.layers.Embedding(input_dim =top_words, output_dim=vector_len)(encode_desc_padding)
embedding.shape

## 임베딩 데이터 셋

In [ ]:
batch_size = 5
def embedding_dataset(data, shuffle=True, batch_size=32):
  ds = tf.data.Dataset.from_tensor_slices({'desc': data})
  if shuffle:
    ds = ds.shuffle(buffer_size=len(data))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [ ]:
embedding_dataset = embedding_dataset(encode_desc_padding, batch_size=batch_size)

In [ ]:
for batch in embedding_dataset.take(1):
  print(batch)

In [ ]:
embedding = tf.keras.layers.Embedding(input_dim =top_words, output_dim=vector_len)

In [ ]:
embedding_layer = lambda feature: embedding(feature)

In [ ]:
embedding_layer(encode_desc_padding).shape

In [ ]:
embedding_dataset.element_spec['desc'].shape[1]

In [ ]:
embedding_col = tf.keras.Input(shape=(), name='desc', dtype='int64')

In [ ]:
def get_text_encoding_layer(name, dataset, top_words, vector_len):
   # Prepare a `tf.data.Dataset` that only yields the feature.
  feature_ds = dataset.map(lambda x: x[name])
  embedding = tf.keras.layers.Embedding(input_dim =top_words, output_dim=vector_len)
  return lambda feature: embedding(feature)

In [ ]:
embedding_layer = get_text_encoding_layer('desc', embedding_dataset, top_words, vector_len)

In [ ]:
embedding_layer(embedding_col)

In [ ]:
all_inputs = []
encoded_features = []

In [ ]:
# text features.
for header in ['desc']:
  embedding_col = tf.keras.Input(shape=(), name='desc', dtype='int64')
  embedding_layer = get_text_encoding_layer('desc', embedding_dataset, top_words, vector_len)
  encoded_embedding_col = embedding_layer(embedding_col)
  all_inputs.append(embedding_col)
  encoded_features.append(encoded_embedding_col)

In [ ]:
for d in all_inputs:
  print(type(d), d.shape)

In [ ]:
for d in encoded_features:
  print(type(d), d.shape)

In [ ]:
from keras.layers import Input, Embedding, Dense
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(64, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
# outputs = Dense(61, activation='softmax', name="Prediction_output")(x)

model2 = tf.keras.Model(all_inputs, output)

model2.summary()

In [ ]:
# optimizer=tf.keras.optimizers.Adam()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

In [ ]:
model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                       optimizer=optimizer,
                       metrics=['accuracy'])

In [ ]:
model2.fit(embedding_dataset,y_train, epochs=10, batch_size=32)

In [ ]:
def df_to_dataset(data, shuffle=True, batch_size=32):
  ds = tf.data.Dataset.from_tensor_slices({'desc': data})
  if shuffle:
    ds = ds.shuffle(buffer_size=len(data))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [ ]:
embedding_dataset = df_to_dataset(encode_desc_padding, batch_size=batch_size)

In [ ]:
encode_desc_padding

In [ ]:
df2 = pd.DataFrame()
df2['desc'] = pre_desc['origin']
df2['y'] = df['key2_encoded']

In [ ]:
y = df2['y'].values

In [ ]:
y

In [ ]:
X = df2['desc']

In [ ]:
# 데이터를 학습용과 검증용으로 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,shuffle=True)

In [ ]:
# 딥러닝 모델을 만들기 위해 categorical 함수 사용
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train, num_classes=61)
y_test_categorical = to_categorical(y_test, num_classes=61)

In [ ]:
X_train

In [ ]:
# train set
train_features = X_train.values
train_labels = y_train_categorical
# features= {
#    'desc': X_train.values,
# }
# labels = y_train_categorical

# tf.data.Dataset 생성
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))

In [ ]:
model2.fit(train_dataset, epochs=10, batch_size=32)

In [ ]:
# test set
predict_features = {
   'desc': X_test.values,
}

In [ ]:
# Apply the preprocessing in tf.data.Dataset.map.
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
dataset = dataset.map(lambda x, y: (preprocessing_model(x), y),
                      num_parallel_calls=tf.data.AUTOTUNE)
# Display a preprocessed input sample.
next(dataset.take(1).as_numpy_iterator())

In [ ]:
pre_desc.head(10)

In [ ]:
# 입력 데이터와 레이블 데이터 준비
train_features = ...
train_labels = ...

# tf.data.Dataset 생성
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))

In [ ]:
df.head()

In [ ]:
df2= df.copy()

In [ ]:
df2